In [3]:
PROJECT_ID = 'dwh-siloam'
REGION = 'asia-southeast1'
print(f"Project ID: {PROJECT_ID}\nRegion: {REGION}")

# Initialize Vertex AI
from pathlib import Path
import vertexai
from google.cloud import aiplatform

print(f"Checking Credentials...")
if not any((Path.cwd()/"service_account").glob('*.json')):
    print("Service account folder is empty. Fallback using default gcloud account")
    aiplatform.init(project=PROJECT_ID, location=REGION)
    vertexai.init(project=PROJECT_ID, location=REGION)
else:
    print('Using service account credentials from service_account folder')
    from google.oauth2 import service_account
    sa_file = list((Path.cwd()/"service_account").glob('*.json'))[0]
    print(f"Using service account file: {sa_file}")
    credentials = service_account.Credentials.from_service_account_file(sa_file)
    aiplatform.init(project=PROJECT_ID, location=REGION, credentials=credentials)
    vertexai.init(project=PROJECT_ID, location=REGION, credentials=credentials)

# Import libraries
from langchain_google_vertexai import VertexAI, ChatVertexAI, create_structured_runnable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

from typing import List, Optional
import requests

from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder
from settings import CopilotSettings
import time

Project ID: dwh-siloam
Region: asia-southeast1
Checking Credentials...
Using service account credentials from service_account folder
Using service account file: /Users/donnymirzaadhitama/workspace/others/chatbot-llm/service_account/dwh-siloam-99402e61edd2.json


In [4]:
config = CopilotSettings()

## Initialize model & prompt template

In [5]:
llm = VertexAI(
    model_name=config.GCP_AGENT_MODEL_NAME, 
    temperature=0, 
    max_output_tokens=8192
)
# chat_llm = ChatVertexAI(
#     model_name=config.GCP_AGENT_MODEL_NAME, 
#     convert_system_message_to_human=True, 
#     temperature=0, 
#     max_output_tokens=8192
# )

In [6]:
prompt = ChatPromptTemplate.from_template(
"""
You are a helpful assistant.
Answer the question below using Indonesian language.
Do not make any assumptions. If you don't know the answer, just answer that you don't know.

{question}
"""
)

In [7]:
chain = (
    prompt
    | llm
)

response = chain.invoke({"question": "Apa makanan kesukaanku?"})
print(response)

 Maaf, saya tidak tahu makanan kesukaan Anda.


## Make a conversation scheme

In [8]:
system_message = """
You are a helpful assistant.
Answer the question below using Indonesian language.
Do not make any assumptions. If you don't know the answer, just answer that you don't know.
"""

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_message
        ),
        MessagesPlaceholder(variable_name="message"),
    ]
)

chain = (
    prompt
    | llm
)

In [10]:
messages = [
    AIMessage(
        content="Halo, apa kabar?",
    ),
    HumanMessage(
        content="Baik."
    ),
    AIMessage(
        content="Ada yang bisa saya bantu?",
    ),
]

query = HumanMessage(
    content="Siapa nama presiden Indonesia tahun 2025?"
)

In [11]:
messages.append(query)
messages

[AIMessage(content='Halo, apa kabar?'),
 HumanMessage(content='Baik.'),
 AIMessage(content='Ada yang bisa saya bantu?'),
 HumanMessage(content='Siapa nama presiden Indonesia tahun 2025?')]

In [12]:
response = chain.invoke({"message": messages})
print(response)

 Maaf, saya tidak dapat menjawab pertanyaan tersebut karena pengetahuan saya terbatas hingga April 2023. Untuk informasi terkini, saya sarankan Anda untuk memeriksa situs web resmi pemerintah Indonesia atau sumber berita yang terpercaya.


In [14]:
answer = response.split("AI:")[-1].strip()

## Start FU conversation here
Every you asked different question, edit `fu_query` variable. And run the cell below again.

In [23]:
fu_query = "Jadi siapakah yang menang pemilu pilpres kemarin?"
response = {"question":"apa itu...", "chat_history":[]}
start = time.time()
answer = AIMessage(
    content=answer
)
query = HumanMessage(
    content=fu_query
)
messages.append(answer)
messages.append(query)

response = chain.invoke({"message": messages})
if "AI:" in response:
    answer = response.split("AI:")[-1].strip()
else:
    answer = response
print(response)
print("Elapsed time:", round(time.time() - start, 3), "[sec]")

 Maaf, pengetahuan saya terbatas hingga April 2023. Untuk informasi terkini, saya sarankan Anda untuk memeriksa situs web resmi pemerintah Indonesia atau sumber berita yang terpercaya.
Elapsed time: 1.435 [sec]


Check the question history here.

In [83]:
len(messages)

52